In [1]:
import pandas as pd
import numpy as np

## Note:
### To run this you'll need an instance with lots of memory. I used ~128GiB and 192GiB

In [2]:
df_raw = pd.read_csv('s3://hype-ml.compassphs.com/cigna_Med_Claim_LineItem.csv', on_bad_lines='skip', sep="|")

/tmp/ipykernel_816/3191982742.py:1: DtypeWarning: Columns (1,49,52,78,80,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('s3://hype-ml.compassphs.com/cigna_Med_Claim_LineItem.csv', on_bad_lines='skip', sep="|")


In [3]:
df = df_raw.copy()
df.head()

,Med_Claim_LineItem_Key,Payer_Claim_Number,Line_Item_Number,Payer_Key,Plan_Type_Key,Payee_Key,Claim_Reporting_Category_Key,Billing_Provider_Key,Rendering_Provider_Key,Parent_Provider_Key,...,Diagnosis_ICD10_1_Key,Diagnosis_ICD10_2_Key,Diagnosis_ICD10_3_Key,Diagnosis_ICD10_4_Key,Diagnosis_ICD10_5_Key,Diagnosis_ICD10_6_Key,RawTableKey,RollUpKey,Claim_Type_Key,Place_Of_Service_Key
0,0,307016093900,0,4,0,0,0,0,0,0,...,21769.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,1,307016093900,0,4,0,0,0,0,0,0,...,21769.0,0.0,0.0,0.0,0.0,0,0,108981130,0,0
2,2,130018979600,0,4,0,0,0,0,0,0,...,68849.0,0.0,0.0,0.0,0.0,0,0,108980541,0,0
3,3,130018979600,0,4,0,0,0,0,0,0,...,68849.0,0.0,0.0,0.0,0.0,0,0,108980542,0,0
4,4,130018979600,0,4,0,0,0,0,0,0,...,68849.0,9350.0,2593.0,0.0,0.0,0,0,108980543,0,0


In [4]:
pd.set_option('display.max_rows', 500)
df.dtypes

Med_Claim_LineItem_Key               int64
Payer_Claim_Number                  object
Line_Item_Number                     int64
Payer_Key                            int64
Plan_Type_Key                        int64
Payee_Key                            int64
Claim_Reporting_Category_Key         int64
Billing_Provider_Key                 int64
Rendering_Provider_Key               int64
Parent_Provider_Key                  int64
Rendering_Provider_Location_Key      int64
Provider_Specialty_Key               int64
Provider_Type_Key                    int64
Place_Of_Treatment_Key               int64
Diagnosis_1_Key                      int64
Diagnosis_2_Key                      int64
Diagnosis_3_Key                      int64
Diagnosis_4_Key                      int64
Diagnosis_5_Key                      int64
Diagnosis_6_Key                      int64
POA_1_Key                            int64
POA_2_Key                            int64
POA_3_Key                            int64
Employer_Ke

In [5]:
df_codes = pd.read_csv('s3://hype-ml.compassphs.com/ICD10.csv', on_bad_lines='skip', sep="|", encoding="ISO-8859-1")
df_codes = df_codes[['DiagnosisKey','DiagnosisCode']]
df_codes.head()

,DiagnosisKey,DiagnosisCode
0,1,A000
1,2,A001
2,3,A009
3,4,A0100
4,5,A0101


In [6]:
df_codes.dtypes

DiagnosisKey      int64
DiagnosisCode    object
dtype: object

In [8]:
df_codes['DiagnosisKey'] = df_codes['DiagnosisKey'].astype('float64')
df_codes.dtypes

DiagnosisKey     float64
DiagnosisCode     object
dtype: object

In [9]:
mapping = dict(zip(df_codes['DiagnosisKey'], df_codes['DiagnosisCode']))
# print(mapping)

In [10]:
for i in range(1,7):
    col_name = f'Diagnosis_ICD10_{i}_Key'
    df[col_name] = df[col_name].map(mapping)

In [11]:
print(df['Diagnosis_ICD10_1_Key'].isna().sum())
print(len(df['Diagnosis_ICD10_1_Key']))

455424
23532358


In [12]:
df['Start_Date_Key']

0           20130201
1           20130201
2           20130105
3           20130105
4           20130105
              ...   
23532353    20230116
23532354    20230116
23532355    20230116
23532356    20230116
23532357    20230116
Name: Start_Date_Key, Length: 23532358, dtype: int64

In [13]:
df_proc = pd.read_csv('s3://hype-ml.compassphs.com/CPT_HCPCS_Procedure.csv', on_bad_lines='skip', sep="|", encoding="ISO-8859-1")
df_proc = df_proc[['CPT_HCPCS_Procedure_Key','CPT_HCPCS_Procedure_Code']]
df_proc.head()

,CPT_HCPCS_Procedure_Key,CPT_HCPCS_Procedure_Code
0,0,NaN
1,1,0001F
2,2,0005F
3,3,00100
4,4,00102


In [14]:
df_proc.dtypes

CPT_HCPCS_Procedure_Key      int64
CPT_HCPCS_Procedure_Code    object
dtype: object

In [15]:
proc_mapping = dict(zip(df_proc['CPT_HCPCS_Procedure_Key'], df_proc['CPT_HCPCS_Procedure_Code']))

In [16]:
df['CPT_HCPCS_Procedure_Key'] = df['CPT_HCPCS_Procedure_Key'].map(mapping)

In [17]:
df['CPT_HCPCS_Procedure_Key']

0              NaN
1              NaN
2           I70211
3            I7025
4            J8417
             ...  
23532353       NaN
23532354       NaN
23532355       NaN
23532356       NaN
23532357       NaN
Name: CPT_HCPCS_Procedure_Key, Length: 23532358, dtype: object

In [18]:
# tuples with dx codes , proc codes
flags = {
    'YES': (['M12151', 'M12152', 'M12159', 'M12161', 'M12162', 'M12169', 'M12551', 'M12552', 'M12559', 'M12561', 'M12562', 'M12569', 'M222X1', 'M222X2', 'M222X9', 'M223X1', 'M223X2', 'M223X9', 'M2240', 'M2241', 'M2242', 'M228X1', 'M228X2', 'M228X9', 'M2290', 'M2291', 'M2292', 'M23202', 'M23205', 'M2340', 'M2341', 'M2342', 'M2350', 'M2350', 'M2350', 'M2350', 'M2350', 'M2351', 'M2352', 'M23601', 'M23602', 'M23609', 'M23611', 'M23612', 'M23619', 'M23621', 'M23622', 'M23629', 'M23631', 'M23632', 'M23639', 'M23641', 'M23642', 'M23649', 'M23671', 'M23672', 'M23679', 'M238X1', 'M238X2', 'M238X9', 'M2390', 'M2391', 'M2392', 'M4800', 'M4807', 'M4808', 'M5000', 'M5001', 'M5003', 'M5020', 'M5021', 'M5023', 'M5030', 'M5031', 'M5033', 'M5104', 'M5105', 'M5124', 'M5125', 'M5126', 'M5127', 'M5134', 'M5135', 'M5144', 'M5145', 'M5146', 'M5147', 'M519', 'M519', 'M9923', 'M9924', 'M9925', 'M9926', 'M9927', 'M9928', 'M9929', 'M9933', 'M9934', 'M9935', 'M9936', 'M9937', 'M9938', 'M9939', 'M9943', 'M9944', 'M9945', 'M9946', 'M9947', 'M9948', 'M9949', 'M9953', 'M9954', 'M9955', 'M9956', 'M9957', 'M9958', 'M9959', 'M9963', 'M9964', 'M9965', 'M9966', 'M9967', 'M9968', 'M9969', 'M9973', 'M9974', 'M9975', 'M9976', 'M9977', 'M9978', 'M9979', 'M5022', 'M5032', 'M5002', 'M4806', 'M4806'],None),
    'physical_therapy': (None, ['97161', '97162', '97163', '97001', '97164', '97002', '97750', '97535', '97530', '97112', '97110', '97140', '97760', '97116', '97012', '97032', '97014', 'G0283', '97033', '97034', '97035', '97018', '97026', '97010', '97113', '97150']),
    'knee_xray': (None, ['73564', '73562', '73560', '73565']),
    'hip_xray': (None, ['73502', '72100', '72170', '73501', '73522', '72110', '73521']),
    'back_xray': (None, ['72100', '72110', '72040', '72050']),
    'back_mri': (None, ['72148', '72141', '72158']),
    'knee_mri': (None, ['73721']),
    'hip_mri': (None, ['73721', '72148']),
    'hip_knee_injection': (None, ['20610', 'J3301', '96372', '20611', 'J1030', 'J1040', 'J1885']),
    'knee_injection': (['M1711','M1712','M1710','M170','M172','M1730','M1731','M1732','M174','M175','M179','M25561','M25562','M25461','M25462','M25661','M25662','R2689','R262','M21169'], ['20610', 'J3301', '96372', '20611', 'J1030', 'J1040', 'J1885']),
    'hip_injection': (['M1611','M1612','M160','M1610','M162','M1630','M1631','M1632','M164','M1650','M1651','M1652','M166','M167','M25551','M25552','M25559','M25651','M25652','R2689','R262','M87050','M87051','M87052','M87059','M87850','M87851','M87852','M87859'], ['20610', 'J3301', '96372', '20611', 'J1030', 'J1040', 'J1885']),
    'spine_injection': (['M545','M5416','M5116','M5417','M5117','M5441','M5442','M5431','M5432','M48061','M48062','M9903','M9904','M9905','M5440','M5430','M4316','M4317'], ['96372', '64483', 'J3301', 'J1885', '64484', 'J1040', 'J1030', '27096']),
    'knee_osteoarthritis': (['M1711', 'M1712', 'M1710',  'M170',  'M172', 'M1730', 'M1731', 'M1732', 'M174', 'M175', 'M179'], None),
    'hip_osteoarthritis': (['M1611', 'M1612', 'M160',  'M1610', 'M162', 'M1630', 'M1631', 'M1632', 'M164', 'M1650', 'M1651', 'M1652', 'M166', 'M167'], None),
    'knee_pain': (['M25561', 'M25562'], None),
    'hip_pain': (['M25551', 'M25552', 'M25559'], None),
    'hip_necrosis': (['M87050', 'M87051', 'M87052', 'M87059', 'M87850', 'M87851', 'M87852', 'M87859'], None),
    'knee_effusion': (['M25461', 'M25462'], None),
    'knee_stiffness': (['M25661', 'M25662'], None),
    'hip_stiffness': (['M25651', 'M25652'], None),
    'gait_abnormality': (['R2689', 'R262'], None),
    'genu_varum': (['M21169'], None),
    'knee_rheumatoid_arthritis': (['M05661', 'M05662', 'M05669', 'M05761', 'M05762', 'M05769', 'M05861', 'M05862', 'M05869', 'M06061', 'M06062', 'M06069', 'M06861', 'M06862', 'M06869'], None),
    'spine_pain': (['M545', 'M549'], None),
    'radiculopathy': (['M5416', 'M5116', 'M5417',   'M5117',  'M5441', 'M5442', 'M5431', 'M5432', 'M5440', 'M5430'], None),
    'stenosis': (['M48061', 'M48062', 'M4807'], None),
    'disc_degeneration': (['M5127'], None),
    'segmental_dysfunction': (['M9903', 'M9901', 'M9904', 'M9905'], None),
    'spondylosis': (['M47896'], None),
    'spondylolisthesis': (['M4316', 'M4317'], None)
}

In [37]:
final_output = df[(df['Start_Date_Key'].astype('string').str[:4] == str(2021))][['Patient_Key']].drop_duplicates().reset_index(drop=True)

In [ ]:
for year in range(2018, 2022):
    current_df = df[df['Start_Date_Key'].astype('string').str[:4] == str(year)][['Patient_Key','Start_Date_Key','Diagnosis_ICD10_1_Key', 'Diagnosis_ICD10_2_Key', 'Diagnosis_ICD10_3_Key', 'Diagnosis_ICD10_4_Key', 'Diagnosis_ICD10_5_Key', 'Diagnosis_ICD10_6_Key', 'CPT_HCPCS_Procedure_Key']]
    year_flags = {}
    grouped_data = current_df.groupby('Patient_Key')
    for group_id, group_data in grouped_data:      
        group_flags = {}
        for flag, (dx_codes, proc_codes) in flags.items():
            if dx_codes is None and proc_codes is None:
                group_flags[flag] = 'No'
            elif dx_codes is None:
                flag_value = np.where(group_data['CPT_HCPCS_Procedure_Key'].isin(proc_codes), 'Yes', 'No')
                group_flags[flag] = 'Yes' if 'Yes' in flag_value else 'No'
            elif proc_codes is None:
                dx_columns = group_data[['Diagnosis_ICD10_1_Key', 'Diagnosis_ICD10_2_Key', 'Diagnosis_ICD10_3_Key', 'Diagnosis_ICD10_4_Key', 'Diagnosis_ICD10_5_Key', 'Diagnosis_ICD10_6_Key']]
                flag_value = np.where(dx_columns.isin(dx_codes).any(axis=1), 'Yes', 'No')
                group_flags[flag] = 'Yes' if 'Yes' in flag_value else 'No'
            else:
                dx_columns = group_data[['Diagnosis_ICD10_1_Key', 'Diagnosis_ICD10_2_Key', 'Diagnosis_ICD10_3_Key', 'Diagnosis_ICD10_4_Key', 'Diagnosis_ICD10_5_Key', 'Diagnosis_ICD10_6_Key']]
                proc_condition = group_data['CPT_HCPCS_Procedure_Key'].isin(proc_codes)
                dx_condition = dx_columns.isin(dx_codes).any(axis=1)
                flag_value = np.where(proc_condition & dx_condition, 'Yes', 'No')
                group_flags[flag] = 'Yes' if 'Yes' in flag_value else 'No'
        year_flags[group_id] = group_flags     
    year_flags_df = pd.DataFrame.from_dict(year_flags, orient='index')
    year_flags_df.columns = [f'{flag}_{year}' for flag in flags.keys()]
    
    final_output = pd.merge(final_output, year_flags_df, how='left', left_on='Patient_Key', right_index=True)
    print("Finished " + str(year))


Finished 2018
Finished 2019
Finished 2020
Finished 2021


In [ ]:
final_output

,Patient_Key,YES_2018,physical_therapy_2018,knee_xray_2018,hip_xray_2018,back_xray_2018,back_mri_2018,knee_mri_2018,hip_mri_2018,hip_knee_mri_2018,...,gait_abnormality_2021,genu_varum_2021,knee_rheumatoid_arthritis_2021,spine_pain_2021,radiculopathy_2021,stenosis_2021,disc_degeneration_2021,segmental_dysfunction_2021,spondylosis_2021,spondylolisthesis_2021
0,11274803,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,16831019,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
2,59821347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No
3,22344577,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,16831016,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158822,85934294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No
158823,66525697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No
158824,66519299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No
158825,66519300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No


In [ ]:
final_output[final_output['YES_2021'] == 'Yes']

,Patient_Key,YES_2018,physical_therapy_2018,knee_xray_2018,hip_xray_2018,back_xray_2018,back_mri_2018,knee_mri_2018,hip_mri_2018,hip_knee_mri_2018,...,gait_abnormality_2021,genu_varum_2021,knee_rheumatoid_arthritis_2021,spine_pain_2021,radiculopathy_2021,stenosis_2021,disc_degeneration_2021,segmental_dysfunction_2021,spondylosis_2021,spondylolisthesis_2021
12,66363871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,Yes,Yes,Yes,No,Yes,No,No
20,8176767,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
60,16830862,No,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,No,No,No,No,No
77,59821287,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
99,59821070,No,No,No,No,No,No,No,No,No,...,Yes,No,No,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158348,85934924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,Yes,Yes,No,No,Yes,No,No
158412,85934723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,No,No,Yes,No,No,No,No,No,No
158442,85935324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,Yes,Yes,No,No,No,No,No
158464,85935574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,No,No,No,No,No,No,No,No


In [ ]:
final_output.to_csv('cigna_inference_test_v2.csv', index=False)